In [10]:
#Jimi at 12:19
def gradientDescentIteration(b_current, theta1_current, theta2_current, points, learningRate):
    b_gradient = 0
    m_gradient = 0
    m_gradient2 = 0
    yIndx = len(data[0])-1 
    N = float(len(points))
    # calculate the gradient for b and m over all the training data points
    for i in range(0, len(points)):
        b_gradient += -(2/N) * (points[i][yIndx] - ((m_current*points[i][0]) + b_current))
        theta_1 += -(2/N) * points[i][1] * (points[i][yIndx] - ((theta1_current * points[i][1]) + b_current))
        theta_2 += -(2/N) * points[i][2] * (points[i][yIndx] - ((theta2_current * points[i][2]) + b_current))
    #gradient update step
    new_b = b_current - (learningRate * b_gradient)
    new_theta1 = theta1_current - (learningRate * m_gradient)
    new_theta2 = theta2_current - (learningRate * m_gradient2)
    return [new_b, new_theta1, new_theta2]

#data in the of tuples [x, y]
data = [[1, 3, 2], [2,20, 3], [3,5, 7], [4, 5, 8], [5, 9, 9]]

b = 0
m = 1
weightVector = [b, m]
#for n gradient iterations refine the weight vector
for i in range(30):
    weightVector = gradientDescentIteration(weightVector[0], 
                                            weightVector[1], 
                                            weightVector[2],
                                            data, 0.01)
    
print "Linear Regression Model is b =  %f m = %f" %(weightVector[0], weightVector[1] ) 

Linear Regression Model is b =  3.779874 m = 56.874214


In [11]:
# From Michael Hoven
def gradientDescentIteration(b_current, m_current, m2_current, points, learningRate):
    b_gradient = 0
    m_gradient = 0
    m2_gradient = 0
    yIndx = len(data[0])-2
    y2Indx = len(data[0])-1 
    N = float(len(points))
    # calculate the gradient for b and m over all the training data points
    for i in range(0, len(points)):
        b_gradient += -(2/N) * (points[i][yIndx] - ((m_current*points[i][0])+(m2_current*points[i][0]) + b_current))
        m_gradient += -(2/N) * points[i][0] * (points[i][yIndx] - ((m_current * points[i][0]) + b_current))
        m2_gradient += -(2/N) * points[i][0] * (points[i][y2Indx] - ((m_current * points[i][0]) + b_current))
    #gradient update step
    new_b = b_current - (learningRate * b_gradient)
    new_m = m_current - (learningRate * m_gradient)
    new_m2 = m2_current - (learningRate * m2_gradient)
    return [new_b, new_m, new_m2]
#data in the of tuples [x, y]
data = [[1, 3, 2], [2,20, 3], [3,5, 7], [4, 5, 8], [5, 9, 9]]
b = 0
m = 1
m2 = 1
weightVector = [b, m, m2]
#for n gradient iterations refine the weight vector
for i in range(30):
    weightVector = gradientDescentIteration(weightVector[0], weightVector[1], weightVector[2], data, 0.0001)
print "Linear Regression Model is b = %f m = %f m2 = %f" %(weightVector[0], weightVector[1], weightVector[2]  )

Linear Regression Model is b = 0.013148 m = 1.078934 m2 = 1.058534


In [ ]:
# from Reed
def stepGradient(b_current, m1_current, m2_current, points, learningrate):
    b_gradient = 0
    m1_gradient = 0
    m2_gradient = 0
    yindx = len(data[0])-1
    N = float(len(points))
    for i in range(0, len(points)):
        b_gradient += -(2/N) * (points[i][yindx] - ((m1_current*points[i][1]) + m2_current*points[i][2] + b_current))
        m1_gradient += -(2/N) * points[i][1] * (points[i][yindx] - ((m1_current * points[i][1] + b_current)))
        m2_gradient += -(2/N) * points[i][2] * (points[i][yindx] - ((m2_current * points[i][2] + b_current)))
        
    new_b = b_current - (learningrate * b_gradient)
    new_m1 = m1_current - (learningrate * m1_gradient)
    new_m2 = m2_current - (learningrate * m2_gradient)
    return [new_b, new_m1, new_m2]
#data in the form of tuples
data = [[1,3, 2], [2,20,3], [3, 5, 7], [4, 5, 8], [5, 9, 9]]
b=0
m1=1
m2=1
weightVector = [b,m1, m2]
for i in range(30):
    weightVector = stepGradient(weightVector[0], weightVector[1], weightVector[2], data, 0.0001)
print "Linear Regression " + str(weightVector[0]) + " " + str(weightVector[1]) + " " + str(weightVector[2])

Adding another variable is adding another m

In [ ]:
from mrjob.job import MRJob

class MRRegression(MRJob):
    
    def mapper1(self): 
        with open('weights.txt', 'r') as f:
            self.weights = [float(v) for v in f.readlines().split(',')]
        self.partial_gradient = [0] * len(self.weights)
        self.partial_count = 0
        
    # MAPPER Calculate partial gradient for each example
    def mapper2(self, _, line):
        D = (map(float, line.split(',')))
        # y_hat is the predicted value given the current weights
        y_hat = self.weights[0] + self.weights[1] * D[1]
        # Update partial gradient vector with gradient from current example
        yield (None, (D[0] - y_hat, D[0] - y_hat) * D[1], 1)
        
    # REDUCER Accumulate parital gradient from mapper and emit total gradient
    # Output: key = None, Value = gradient vector
    def reducer(self, _, partial_gradient_record):
        total_gradient = [0] * 2
        total_connt = 0
        for partial_gradient, partial_count in partial_gradient_record:
            total_count = total_count + partial_count
            total_gradient[0] = total_gradient[0] + partial_gradient[0]
            total_gradient[1] = total_gradient[] + partial_gradient[]
            